In [46]:
import gymnasium as gym
import gym_donkeycar

import os
import ray
import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from preprocessing import grayscale_transform as transform

torch.cuda.empty_cache()
%matplotlib inline

In [47]:
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune import register_env

In [62]:
exe_path = "/home/till/Desktop/Thesis/donkeycar_sim/DonkeySimLinux/donkey_sim.x86_64"
port = 9091
config = {
    "exe_path" : exe_path, 
    "port" : port
}
env = gym.make(
    "GymV21Environment-v0", 
    env_id="donkey-generated-roads-v0", # donkey-minimonaco-track-v0 
    max_episode_steps=100,
    make_kwargs={
        "conf": config
    })

starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0
donkey subprocess started
Found path: /home/till/Desktop/Thesis/donkeycar_sim/DonkeySimLinux/donkey_sim.x86_64


loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


closing donkey sim subprocess


## SB3

In [60]:
import gymnasium as gym

from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments
# env = make_vec_env("CartPole-v1", n_envs=1)

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)
# model.save("a2c_cartpole")

# del model # remove to demonstrate saving and loading

# model = A2C.load("a2c_cartpole")

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: The algorithm only supports (<class 'gym.spaces.box.Box'>, <class 'gym.spaces.discrete.Discrete'>, <class 'gym.spaces.multi_discrete.MultiDiscrete'>, <class 'gym.spaces.multi_binary.MultiBinary'>) as action spaces but Box(0.0, 1.0, (1,), float32) was provided

## Ray

In [34]:
type(env)

gymnasium.wrappers.time_limit.TimeLimit

In [27]:
# Initialize Ray
ray.init()

2023-05-07 23:39:49,244	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.10.4
Ray version:,2.4.0


In [42]:
register_env("custom_env", lambda config: env)

TypeError: ray.cloudpickle.dumps(<function <lambda> at 0x7fae1195ce50>) failed.
To check which non-serializable variables are captured in scope, re-run the ray script with 'RAY_PICKLE_VERBOSE_DEBUG=1'.

In [41]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print



algo = (
    PPOConfig()
    .rollouts(num_rollout_workers=1)
    .resources(num_gpus=1)
    .environment(env=wrap_env(env)
)
    .build()
)

for i in range(10):
    result = algo.train()
    print(pretty_print(result))

    if i % 5 == 0:
        checkpoint_dir = algo.save()
        print(f"Checkpoint saved in directory {checkpoint_dir}")

ImportError: cannot import name 'wrap_env' from 'ray.rllib.env' (/home/till/.local/lib/python3.10/site-packages/ray/rllib/env/__init__.py)

In [35]:
# Define the training configuration
config = PPOConfig("GymV21Environment-v0")
config["num_workers"] = 1
config["framework"] = "torch"  # or "tf" for TensorFlow
config["env"] = "GymV21Environment-v0"
config["env_config"] = {
    "env_id": "donkey-generated-roads-v0",
    "max_episode_steps": 100,
    "make_kwargs": {
        "conf": {
            "exe_path": "/home/till/Desktop/Thesis/donkeycar_sim/DonkeySimLinux/donkey_sim.x86_64",
            "port": 9091
        }
    }
}

# Create the RLlib trainer
trainer = ppo.PPOTrainer(config=config)

# Training loop
for i in range(10):  # Replace 10 with the desired number of training iterations
    result = trainer.train()
    print(f"Iteration {i}: {result}")

# Evaluate the trained agent
env = gym.make("GymV21Environment-v0", env_id="donkey-generated-roads-v0", make_kwargs={"conf": config["env_config"]["make_kwargs"]["conf"]})
obs = env.reset()
done = False
total_reward = 0
while not done:
    action = trainer.compute_action(obs)
    obs, reward, done, _ = env.step(action)
    total_reward += reward

print(f"Total reward: {total_reward}")

AttributeError: can't set attribute 'num_workers'

In [26]:
# Shut down Ray
ray.shutdown()

In [16]:
config = (  # 1. Configure the algorithm,
    PPOConfig()
    .environment("GymV21Environment-v0")
    .rollouts(num_rollout_workers=1)
    .framework("torch")
    .training(model={"fcnet_hiddens": [64, 64]})
    .evaluation(evaluation_num_workers=1)
)

algo = config.build()  # 2. build the algorithm,

2023-05-07 23:34:32,904	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/till/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_32296/3739248578.py", line 10, in <module>
    algo = config.build()  # 2. build the algorithm,
  File "/home/till/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm_config.py", line 1071, in build
    return algo_class(
  File "/home/till/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 392, in __init__
    config.validate()
  File "/home/till/.local/lib/python3.10/site-packages/ray/rllib/algorithms/ppo/ppo.py", line 274, in validate
    super().validate()
  File "/home/till/.local/lib/python3.10/site-packages/ray/rllib/algorithms/pg/pg.py", line 99, in validate
    super().validate()
  File "/home/till/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm_config.py", line 957, in validate
    self.is_atari 

In [9]:
for _ in range(5):
    print(algo.train())  # 3. train it,

2023-05-07 23:26:20,973	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 4.999999873689376e-05, 'total_loss': 9.945138, 'policy_loss': -0.00228693, 'vf_loss': 9.947379, 'vf_explained_var': -2.1112524e-05, 'kl': 0.00022546813, 'entropy': 1.7915361, 'entropy_coeff': 0.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 480.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': -686.0, 'episode_reward_min': -884.0, 'episode_reward_mean': -785.9, 'episode_len_mean': 200.0, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-821.0, -767.0, -830.0, -686.0, -839.0, -740.0, -848.0, -785.0,

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.10000000149011612, 'cur_lr': 4.999999873689376e-05, 'total_loss': 9.900982, 'policy_loss': -0.01639441, 'vf_loss': 9.916453, 'vf_explained_var': -0.00028671778, 'kl': 0.009232815, 'entropy': 1.7767756, 'entropy_coeff': 0.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 1440.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}}, 'num_env_steps_sampled': 8000, 'num_env_steps_trained': 8000, 'num_agent_steps_sampled': 8000, 'num_agent_steps_trained': 8000}, 'sampler_results': {'episode_reward_max': -641.0, 'episode_reward_min': -920.0, 'episode_reward_mean': -771.275, 'episode_len_mean': 200.0, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-821.0, -767.0, -830.0, -686.0, -839.0, -740.0, -848.0, -785.0

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.10000000149011612, 'cur_lr': 4.999999873689376e-05, 'total_loss': 9.861151, 'policy_loss': -0.013795248, 'vf_loss': 9.874287, 'vf_explained_var': -0.00025686945, 'kl': 0.0065987557, 'entropy': 1.7391791, 'entropy_coeff': 0.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 2400.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 12000, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 12000}, 'sampler_results': {'episode_reward_max': -115.0, 'episode_reward_min': -920.0, 'episode_reward_mean': -706.2666666666667, 'episode_len_mean': 196.16666666666666, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-821.0, -767.0, -830.0, -686.0, -

KeyboardInterrupt: 

In [ ]:
algo.evaluate()  # 4. and evaluate it.